# Chapter 8: ニューラルネット

第6章で取り組んだニュース記事のカテゴリ分類を題材として，ニューラルネットワークでカテゴリ分類モデルを実装する．なお，この章ではPyTorch, TensorFlow, Chainerなどの機械学習プラットフォームを活用せよ．

## 70. 単語ベクトルの和による特徴量

問題50で構築した学習データ，検証データ，評価データを行列・ベクトルに変換したい．例えば，学習データについて，すべての事例$x_i$の特徴ベクトル$\boldsymbol{x}_i$を並べた行列$X$と，正解ラベルを並べた行列（ベクトル）$Y$を作成したい．

$$
X = \begin{pmatrix} 
  \boldsymbol{x}_1 \\ 
  \boldsymbol{x}_2 \\ 
  \dots \\ 
  \boldsymbol{x}_n \\ 
\end{pmatrix} \in \mathbb{R}^{n \times d},
Y = \begin{pmatrix} 
  y_1 \\ 
  y_2 \\ 
  \dots \\ 
  y_n \\ 
\end{pmatrix} \in \mathbb{N}^{n}
$$

ここで，$n$は学習データの事例数であり，$\boldsymbol{x}_i \in \mathbb{R}^d$と$y_i \in \mathbb{N}$はそれぞれ，$i \in \{1, \dots, n\}$番目の事例の特徴量ベクトルと正解ラベルを表す．
なお，今回は「ビジネス」「科学技術」「エンターテイメント」「健康」の4カテゴリ分類である．$\mathbb{N}_{<4}$で$4$未満の自然数（$0$を含む）を表すことにすれば，任意の事例の正解ラベル$y_i$は$y_i \in \mathbb{N}_{<4}$で表現できる．
以降では，ラベルの種類数を$L$で表す（今回の分類タスクでは$L=4$である）．

$i$番目の事例の特徴ベクトル$\boldsymbol{x}_i$は，次式で求める．

$$
\boldsymbol{x}_i = \frac{1}{T_i} \sum_{t=1}^{T_i} \mathrm{emb}(w_{i,t})
$$

ここで，$i$番目の事例は$T_i$個の（記事見出しの）単語列$(w_{i,1}, w_{i,2}, \dots, w_{i,T_i})$から構成され，$\mathrm{emb}(w) \in \mathbb{R}^d$は単語$w$に対応する単語ベクトル（次元数は$d$）である．すなわち，$i$番目の事例の記事見出しを，その見出しに含まれる単語のベクトルの平均で表現したものが$\boldsymbol{x}_i$である．今回は単語ベクトルとして，問題60でダウンロードしたものを用いればよい．$300$次元の単語ベクトルを用いたので，$d=300$である．

$i$番目の事例のラベル$y_i$は，次のように定義する．

$$
y_i = \begin{cases}
0 & (\mbox{記事}x_i\mbox{が「ビジネス」カテゴリの場合}) \\
1 & (\mbox{記事}x_i\mbox{が「科学技術」カテゴリの場合}) \\
2 & (\mbox{記事}x_i\mbox{が「エンターテイメント」カテゴリの場合}) \\
3 & (\mbox{記事}x_i\mbox{が「健康」カテゴリの場合}) \\
\end{cases}
$$

なお，カテゴリ名とラベルの番号が一対一で対応付いていれば，上式の通りの対応付けでなくてもよい．

以上の仕様に基づき，以下の行列・ベクトルを作成し，ファイルに保存せよ．

+ 学習データの特徴量行列: $X_{\rm train} \in \mathbb{R}^{N_t \times d}$
+ 学習データのラベルベクトル: $Y_{\rm train} \in \mathbb{N}^{N_t}$
+ 検証データの特徴量行列: $X_{\rm valid} \in \mathbb{R}^{N_v \times d}$
+ 検証データのラベルベクトル: $Y_{\rm valid} \in \mathbb{N}^{N_v}$
+ 評価データの特徴量行列: $X_{\rm test} \in \mathbb{R}^{N_e \times d}$
+ 評価データのラベルベクトル: $Y_{\rm test} \in \mathbb{N}^{N_e}$

なお，$N_t, N_v, N_e$はそれぞれ，学習データの事例数，検証データの事例数，評価データの事例数である．

In [10]:
import pandas as pd
import numpy as np

In [4]:
train_df = pd.read_table('../data/news_aggregator/train.txt')
valid_df = pd.read_table('../data/news_aggregator/valid.txt')
test_df = pd.read_table('../data/news_aggregator/test.txt')

In [6]:
categories = ['b', 't', 'e', 'm']

In [7]:
from gensim.models import KeyedVectors

In [8]:
model = KeyedVectors.load_word2vec_format('../data/google_news.bin.gz', binary=True)

In [115]:
# tokenize, make lowercase and make stem

import re
import string

def tokenize(x):
    table = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
    x = x.translate(table).split()
    x = [w.lower() for w in x]
    return x

In [116]:
train_df['tokens'] = train_df['title'].map(tokenize)
valid_df['tokens'] = valid_df['title'].map(tokenize)
test_df['tokens'] = test_df['title'].map(tokenize)

In [117]:
def labelize(x):
    return categories.index(x)

In [118]:
train_df['label'] = train_df['category'].map(labelize)
valid_df['label'] = valid_df['category'].map(labelize)
test_df['label'] = test_df['category'].map(labelize)

In [124]:
import torch

def vectorize(tokens):
    vec = np.array([model[w] for w in tokens if w in model])
    return torch.tensor(np.mean(vec, axis=0))

In [125]:
X_train = torch.stack([vectorize(tokens) for tokens in train_df['tokens']])
X_valid = torch.stack([vectorize(tokens) for tokens in valid_df['tokens']])
X_test = torch.stack([vectorize(tokens) for tokens in test_df['tokens']])

In [122]:
y_train = torch.tensor(train_df['label'])
y_valid = torch.tensor(valid_df['label'])
y_test = torch.tensor(test_df['label'])

In [129]:
torch.save(X_train, '../data/X_train.pt')
torch.save(X_valid, '../data/X_valid.pt')
torch.save(X_test, '../data/X_test.pt')
torch.save(y_train, '../data/y_train.pt')
torch.save(y_valid, '../data/y_valid.pt')
torch.save(y_test, '../data/y_test.pt')

## 71. 単層ニューラルネットワークによる予測

問題70で保存した行列を読み込み，学習データについて以下の計算を実行せよ．

$$
\hat{\boldsymbol{y}}_1 = {\rm softmax}(\boldsymbol{x}_1 W), \\
\hat{Y} = {\rm softmax}(X_{[1:4]} W)
$$

ただし，${\rm softmax}$はソフトマックス関数，$X_{[1:4]} \in \mathbb{R}^{4 \times d}$は特徴ベクトル$\boldsymbol{x}_1, \boldsymbol{x}_2, \boldsymbol{x}_3, \boldsymbol{x}_4$を縦に並べた行列である．

$$
X_{[1:4]} = \begin{pmatrix} 
  \boldsymbol{x}_1 \\ 
  \boldsymbol{x}_2 \\ 
  \boldsymbol{x}_3 \\ 
  \boldsymbol{x}_4 \\ 
\end{pmatrix}
$$

行列$W \in \mathbb{R}^{d \times L}$は単層ニューラルネットワークの重み行列で，ここではランダムな値で初期化すればよい（問題73以降で学習して求める）．なお，$\hat{\boldsymbol{y}}_1 \in \mathbb{R}^L$は未学習の行列$W$で事例$x_1$を分類したときに，各カテゴリに属する確率を表すベクトルである．
同様に，$\hat{Y} \in \mathbb{R}^{n \times L}$は，学習データの事例$x_1, x_2, x_3, x_4$について，各カテゴリに属する確率を行列として表現している．

In [134]:
X_train = torch.load('../data/X_train.pt')
X_valid = torch.load('../data/X_valid.pt')
X_test = torch.load('../data/X_test.pt')
y_train = torch.load('../data/y_train.pt')
y_valid = torch.load('../data/y_valid.pt')
y_test = torch.load('../data/y_test.pt')

In [135]:
from torch import nn

class SLPNet(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.fc = nn.Linear(input_size, output_size, bias=False)
        nn.init.normal_(self.fc.weight, 0.0, 1.0)
    
    def forward(self, x):
        x = self.fc(x)
        return x

In [187]:
model = SLPNet(300, 4)
y_hat_1 = torch.softmax(model(X_train[:1]), dim=-1)
y_hat_1

tensor([[0.0467, 0.3497, 0.5099, 0.0937]], grad_fn=<SoftmaxBackward>)

In [188]:
Y_hat = torch.softmax(model(X_train[:4]), dim=-1)
Y_hat

tensor([[0.0467, 0.3497, 0.5099, 0.0937],
        [0.1622, 0.3983, 0.2954, 0.1442],
        [0.1508, 0.0474, 0.1624, 0.6395],
        [0.0488, 0.4084, 0.4985, 0.0443]], grad_fn=<SoftmaxBackward>)

## 72. 損失と勾配の計算

学習データの事例$x_1$と事例集合$x_1, x_2, x_3, x_4$に対して，クロスエントロピー損失と，行列$W$に対する勾配を計算せよ．なお，ある事例$x_i$に対して損失は次式で計算される．

$$
l_i = - \log [\mbox{事例}x_i\mbox{が}y_i\mbox{に分類される確率}]
$$

ただし，事例集合に対するクロスエントロピー損失は，その集合に含まれる各事例の損失の平均とする．

In [197]:
criterion = nn.CrossEntropyLoss()

In [202]:
loss_1 = criterion(model(X_train[:1]), y_train[:1])
model.zero_grad()
loss_1.backward()

print(f'損失: {loss_1}')
print(f'勾配: {model.fc.weight.grad}')

損失: 3.0635852813720703
勾配: tensor([[ 0.0291, -0.0393, -0.0438,  ..., -0.0393, -0.0044, -0.0456],
        [-0.0107,  0.0144,  0.0160,  ...,  0.0144,  0.0016,  0.0167],
        [-0.0156,  0.0210,  0.0234,  ...,  0.0210,  0.0024,  0.0244],
        [-0.0029,  0.0039,  0.0043,  ...,  0.0039,  0.0004,  0.0045]])


In [203]:
loss = criterion(model(X_train[:4]), y_train[:4])
model.zero_grad()
loss.backward()

print(f'損失: {loss}')
print(f'勾配: {model.fc.weight.grad}')

損失: 2.430011034011841
勾配: tensor([[ 0.0139, -0.0235, -0.0299,  ..., -0.0181, -0.0168, -0.0144],
        [-0.0091,  0.0140,  0.0132,  ...,  0.0035,  0.0043,  0.0082],
        [ 0.0165, -0.0174,  0.0093,  ...,  0.0334,  0.0296, -0.0129],
        [-0.0214,  0.0269,  0.0074,  ..., -0.0188, -0.0172,  0.0190]])


## 73. 確率的勾配降下法による学習

確率的勾配降下法（SGD: Stochastic Gradient Descent）を用いて，行列Wを学習せよ．なお，学習は適当な基準で終了させればよい（例えば「100エポックで終了」など）．

In [207]:
from torch.utils.data import Dataset, DataLoader

In [222]:
class NewsDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]

In [231]:
dataset_train = NewsDataset(X_train, y_train)
dataset_valid = NewsDataset(X_valid, y_valid)
dataset_test = NewsDataset(X_test, y_test)

In [232]:
dataloader_train = DataLoader(dataset_train, batch_size=1, shuffle=True)
dataloader_valid = DataLoader(dataset_valid, batch_size=len(dataset_valid), shuffle=False)
dataloader_test = DataLoader(dataset_test, batch_size=len(dataset_test), shuffle=False)

In [237]:
model = SLPNet(300, 4)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-1)

In [238]:
num_epochs = 100

In [239]:
for epoch in range(num_epochs):
    model.train()
    loss_train = 0.0
    for i, (inputs, labels) in enumerate(dataloader_train):
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        loss_train += loss.item()
    
    loss_train = loss_train / i
    
    model.eval()
    with torch.no_grad():
        inputs, labels = next(iter(dataloader_valid))
        outputs = model(inputs)
        loss_valid = criterion(outputs, labels)
        
    print(f'epoch: {epoch + 1}, loss_train: {loss_train:.4f}, loss_valid: {loss_valid:.4f}')  

epoch: 1, loss_train: 0.5370, loss_valid: 0.3871
epoch: 2, loss_train: 0.3689, loss_valid: 0.3597
epoch: 3, loss_train: 0.3374, loss_valid: 0.3456
epoch: 4, loss_train: 0.3206, loss_valid: 0.3393
epoch: 5, loss_train: 0.3093, loss_valid: 0.3351
epoch: 6, loss_train: 0.3007, loss_valid: 0.3340
epoch: 7, loss_train: 0.2948, loss_valid: 0.3291
epoch: 8, loss_train: 0.2894, loss_valid: 0.3300
epoch: 9, loss_train: 0.2856, loss_valid: 0.3285
epoch: 10, loss_train: 0.2820, loss_valid: 0.3282
epoch: 11, loss_train: 0.2796, loss_valid: 0.3296
epoch: 12, loss_train: 0.2774, loss_valid: 0.3293
epoch: 13, loss_train: 0.2747, loss_valid: 0.3282
epoch: 14, loss_train: 0.2730, loss_valid: 0.3266
epoch: 15, loss_train: 0.2715, loss_valid: 0.3286
epoch: 16, loss_train: 0.2700, loss_valid: 0.3313
epoch: 17, loss_train: 0.2686, loss_valid: 0.3305
epoch: 18, loss_train: 0.2669, loss_valid: 0.3322
epoch: 19, loss_train: 0.2663, loss_valid: 0.3311
epoch: 20, loss_train: 0.2647, loss_valid: 0.3371
epoch: 21

## 74. 正解率の計測

問題73で求めた行列を用いて学習データおよび評価データの事例を分類したとき，その正解率をそれぞれ求めよ．

In [245]:
def calculate_accuracy(model, loader):
    model.eval()
    total = 0
    correct = 0
    with torch.no_grad():
        for inputs, labels in loader:
            outputs = model(inputs)
            pred = torch.argmax(outputs, dim=-1)
            total += len(inputs)
            correct += (pred == labels).sum().item()

    return correct / total

In [247]:
acc_train = calculate_accuracy(model, dataloader_train)
acc_test = calculate_accuracy(model, dataloader_test)
print(f'train accuracy: {acc_train:.3f}')
print(f'test accuracy: {acc_test:.3f}')

train accuracy: 0.916
test accuracy: 0.894


## 75. 損失と正解率のプロット

問題73のコードを改変し，各エポックのパラメータ更新が完了するたびに，訓練データでの損失，正解率，検証データでの損失，正解率をグラフにプロットし，学習の進捗状況を確認できるようにせよ．